<a href="https://colab.research.google.com/github/dauparas/tensorflow_examples/blob/master/beta_vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import regularizers

%matplotlib inline
plt.style.use('ggplot')

In [0]:
N_points = 501
# N_points=n_points**3
np.random.seed(1)

#Number of examples
noise_level = 0.005
#Line
t = np.random.uniform(1, 2, N_points)
s = np.random.uniform(1, 2, N_points)
q = np.random.uniform(1, 2, N_points)


# t =np.linspace(0,1, n_points)
# s =np.linspace(0,1, n_points)
# q = np.linspace(0,1, n_points)

# ts = np.asarray(np.meshgrid(t,s,q))
# t = np.reshape(ts[0,:,:,:],[-1])
# s = np.reshape(ts[1,:,:,:],[-1])
# q = np.reshape(ts[2,:,:,:],[-1])


# t=np.sort(t)
# s=np.sort(s)

x_t1 = 1.1+5*0.1*t+noise_level*np.random.randn(N_points) #########t
x_t2 = 1.1+5*0.2*t*t+noise_level*np.random.randn(N_points) #########t
x_t3 = 1.1+0.15*s+noise_level*np.random.randn(N_points) #######
x_t4 = 1.1+0.25*s*s+noise_level*np.random.randn(N_points) #########
x_t5 = 1.1+1.27*q*q+noise_level*np.random.randn(N_points) #######
x_t6 = 1.1+1.67*q+noise_level*np.random.randn(N_points) #######

# x_t1 = np.array(range(N))
# x_t2 = np.array(range(N))
# x_t3 = np.array(range(N))
# x_t4 = np.array(range(N))
# x_t5 = np.array(range(N))
# x_t6 = np.array(range(N))
# x_t7 = np.array(range(N))
# x_t8 = np.array(range(N))

x_t1 = x_t1.reshape(-1,1)
x_t2 = x_t2.reshape(-1,1)
x_t3 = x_t3.reshape(-1,1)
x_t4 = x_t4.reshape(-1,1)
x_t5 = x_t5.reshape(-1,1)
x_t6 = x_t6.reshape(-1,1)
my_data = np.concatenate([x_t1, x_t2, x_t3, x_t4, x_t5, x_t6], axis=1)

#Create training and testing sets
train_data = my_data

#Rescale training data
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

train_data = (train_data - mean) /std

train = train_data

In [0]:
plt.scatter(t,s);
plt.axis('equal');

In [0]:
plt.scatter(t,q);
plt.axis('equal');

In [0]:
plt.scatter(train[:,0], train[:,1])

In [0]:
plt.scatter(train[:,2], train[:,3])

In [0]:
plt.scatter(train[:,4], train[:,5])

In [0]:
angle = np.pi/4
sin = np.sin(angle)
cos = np.cos(angle)

In [0]:
C = np.matmul(train.T, train)
D = np.matmul(train, train.T)

In [0]:
plt.matshow(np.abs(C));
plt.grid(False)
plt.colorbar()

In [0]:
train.shape

In [0]:
# def encoder(x_in):
#     with tf.variable_scope("encoder", reuse=None):
#         e2 = tf.layers.dense(x_in, units=12, activation=tf.nn.leaky_relu)
#         b2 = tf.layers.batch_normalization(e2)
#         e3 = tf.layers.dense(b2, units=12, activation=tf.nn.leaky_relu)
#         b3 = tf.layers.batch_normalization(e3)
#         e4 = tf.layers.dense(b3, units=6, activation=tf.nn.leaky_relu)
#         b4 = tf.layers.batch_normalization(e4)
#         enc_m = tf.layers.dense(b4, units=3)
#         enc_l = tf.layers.dense(b4, units=3)
#         rotation = tf.constant([[1.0, 0, 0],[0, 1, 0], [0,0,1]])
#         enc_mean = tf.einsum('ij, kj->ki', rotation, enc_m)
#         enc_log_sd = tf.einsum('ij, kj->ki', rotation, enc_l)
        
        
#         epsilon = tf.random_normal(tf.shape(enc_mean)) 
#         z  = enc_mean + tf.multiply(epsilon, tf.exp(enc_log_sd))
#         return z, enc_mean, enc_log_sd, enc_m, enc_l

def fully_connected(inputs, out_dim, scope_name='fc', reg_param=10.0, non_linearity=True):
    '''
    A fully connected linear layer
    '''
    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        in_dim = inputs.shape[-1]
        w = tf.get_variable('weights', [in_dim, out_dim],
                            initializer=tf.glorot_uniform_initializer(), regularizer=tf.contrib.layers.l2_regularizer(reg_param, scope=scope_name+'_reg'))
        b = tf.get_variable('biases', [out_dim],
                            initializer=tf.constant_initializer(0.0))
        out = tf.matmul(inputs, w) + b
        if non_linearity:
          return tf.nn.leaky_relu(out)
        else:
          return out
      
      
def encoder(x_in):
    with tf.variable_scope("encoder", reuse=None):
#         e2 = tf.layers.dense(x_in, units=12, activation=tf.nn.leaky_relu)
#         b2 = tf.layers.batch_normalization(e2)
#         e3 = tf.layers.dense(b2, units=12, activation=tf.nn.leaky_relu)
#         b3 = tf.layers.batch_normalization(e3)
#         e4 = tf.layers.dense(b3, units=6, activation=tf.nn.leaky_relu)
#         b4 = tf.layers.batch_normalization(e4)
        w1 = tf.get_variable('w1', [6, 3], initializer=tf.glorot_uniform_initializer())
        b1 = tf.get_variable('b1', [3], initializer=tf.constant_initializer(0.0))
        enc_mean = tf.matmul(x_in, w1)+b1
      
      
        w2 = tf.get_variable('w2', [6, 3], initializer=tf.glorot_uniform_initializer())
        b2 = tf.get_variable('b2', [3], initializer=tf.constant_initializer(0.0))
        enc_log_sd = tf.matmul(x_in, w2)+b2

#         rotation = tf.constant([[1.0, 0, 0],[0, 1, 0], [0,0,1]])
#         enc_mean = tf.einsum('ij, kj->ki', rotation, enc_m)
#         enc_log_sd = tf.einsum('ij, kj->ki', rotation, enc_l)
        
        
        epsilon = tf.random_normal(tf.shape(enc_mean)) 
        z  = enc_mean + tf.multiply(epsilon, tf.exp(enc_log_sd))
        return z, enc_mean, enc_log_sd, w1, w2
# def decoder(z_in):
#     with tf.variable_scope("decoder", reuse=tf.AUTO_REUSE):
#         d0 = tf.layers.dense(z_in, units=3)
#         b0 = tf.layers.batch_normalization(d0)
#         d1 = tf.layers.dense(b0, units=6)
#         b1 = tf.layers.batch_normalization(d1)
#         d2 = tf.layers.dense(b1, units=12)
#         b2 = tf.layers.batch_normalization(d2)
#         dec_mean = tf.layers.dense(b2, units=12)
#         dec_log_sd = tf.layers.dense(b2, units=12)
#         return dec_mean, dec_log_sd
      
def decoder(z_in):
    with tf.variable_scope("decoder", reuse=tf.AUTO_REUSE):
#         d0 = tf.layers.dense(z_in, units=12)

        w3 = tf.get_variable('w3', [3, 6], initializer=tf.glorot_uniform_initializer())
        b3 = tf.get_variable('b3', [6], initializer=tf.constant_initializer(0.0))
        dec_mean = tf.matmul(z_in, w3)+b3
    
      
        return dec_mean, w3

In [0]:
import time
tf.reset_default_graph() #reset the graph 
batch_size = tf.placeholder(tf.int64, name='batch')
repeat_n = tf.placeholder(tf.int64, name='repeat')
n_epochs =1000
BATCH_SIZE = 101
# Wrapping all together -> Switch between train and test set using Initializable iterator
# using two numpy arrays
train = train.astype(np.float32)

Z1 = np.linspace(-0.5,0.5,31)
Z2 = np.linspace(-0.5,0.5,31) 
Z3 = np.linspace(-0.5, 0.5, 31)
Z_test = np.meshgrid(Z1, Z2, Z3)
Z_test = np.asarray(Z_test, dtype=np.float32)
print(Z_test.shape)
Z_test = np.reshape(Z_test, [3, -1])
print(Z_test.shape)
Z_test = Z_test.T


z_test = tf.placeholder(tf.float32, shape=Z_test.shape, name='z_test')

train_data = (train, train)
train_data = tf.data.Dataset.from_tensor_slices(train_data)
train_data = train_data.shuffle(buffer_size=10000)
train_data = train_data.repeat(repeat_n)
train_data = train_data.batch(batch_size)

iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
features, labels = iterator.get_next()
train_init = iterator.make_initializer(train_data)	# initializer for train_data

n_latent = 3
z_sampled, enc_mean, enc_log_sd, w1_e, w2_e = encoder(features)
dec_mean, w1_d = decoder(z_sampled)

dec_mean_o, _ = decoder(z_test)

from __future__ import division


dec_loss = tf.reduce_mean(tf.reduce_sum(tf.square(features-dec_mean),1))

my_c = tf.placeholder(tf.float32, name='my_c')
gamma = 0.0000005
w_loss = 0.1*(tf.nn.l2_loss(w1_e)+tf.nn.l2_loss(w2_e)+tf.nn.l2_loss(w1_d))

latent_loss = 0.1*tf.reduce_mean(tf.abs(enc_mean[:,0])+tf.abs(enc_mean[:,1])+tf.abs(enc_mean[:,2]))
kl_loss = tf.reduce_mean(-0.5 * tf.reduce_sum(1.0+ 2.0*enc_log_sd - tf.square(enc_mean) - tf.exp(2.0 * enc_log_sd), 1))
loss = dec_loss+gamma*tf.abs(kl_loss-my_c)+w_loss+latent_loss

optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

In [0]:
saver = tf.train.Saver()
with tf.Session() as sess:
   
    start_time = time.time()
    sess.run(tf.global_variables_initializer())

    # train the model n_epochs times
    for i in range(n_epochs):
        if i < 1000:
          if i % 10 ==0:
            C_in = 5.0*(1.0*i)/1000
        sess.run(train_init, feed_dict={batch_size: BATCH_SIZE, repeat_n: 1, my_c: C_in})
        total_loss = 0
        total_loss_dec = 0
        total_loss_kl = 0
        total_loss_w = 0
        n_batches = 0
        try:
            while True:
                _, l, l_dec, l_kl, l_w= sess.run([optimizer, loss, dec_loss, kl_loss, w_loss], feed_dict={my_c: C_in})
                total_loss += l
                total_loss_w += l_w
                total_loss_dec += l_dec
                total_loss_kl += l_kl
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        if i % 50 == 0:
          print('Loss {0}: {1}, Dec_loss: {2}, KL_loss: {3}, W loss: {4}'.format(i, total_loss/n_batches, total_loss_dec/n_batches, total_loss_kl/n_batches, total_loss_w/n_batches ))
    print('Total time: {0} seconds'.format(time.time() - start_time))
    
    
    sess.run(train_init, feed_dict={batch_size: train.shape[0], repeat_n: 1})
    try:
      while True:
        dec_mean_out = sess.run([dec_mean])
    except tf.errors.OutOfRangeError:
      pass
    
    
    sess.run(train_init, feed_dict={batch_size: train.shape[0], repeat_n: 1})
    try:
      while True:
        enc_mean_out, enc_log_sd_out = sess.run([enc_mean, enc_log_sd])
    except tf.errors.OutOfRangeError:
      pass

    
#     sess.run(train_init, feed_dict={batch_size: train.shape[0], repeat_n: 1})
#     try:
#       while True:
#         enc_m_out, enc_l_out = sess.run([enc_m, enc_l])
#     except tf.errors.OutOfRangeError:
#       pass
    
    dec_mean_out_i1 = sess.run([dec_mean_o], feed_dict={z_test: Z_test})
    w1_eo, w2_eo, w1_do = sess.run([w1_e, w2_e, w1_d])
#     dec_mean_out_i2, _ = sess.run([dec_mean_o, dec_log_sd_o], feed_dict={z_test: Z2o})
    
    save_path = saver.save(sess, "./ae.ckpt")
    print("Model saved in path: %s" % save_path)
#     # test the model
#     sess.run(test_init, feed_dict={batch_size: test.shape[0]})			# drawing samples from test_data
#     try:
#         while True:
#           z2, p2  = sess.run([z, out])
#     except tf.errors.OutOfRangeError:
#         pass

In [0]:
tf.reset_default_graph()

# Create some variables.
w1_e = tf.get_variable('encoder/w1', shape=[6,3], dtype=tf.float32)
b1_e = tf.get_variable('encoder/b1', shape=[3], dtype=tf.float32)
w1_d = tf.get_variable('decoder/w3', shape=[3,6], dtype=tf.float32)
b1_d = tf.get_variable('decoder/b3', shape=[6], dtype=tf.float32)
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "./ae.ckpt")
  print("Model restored.")
  # Check the values of the variables
  w1_eo = w1_e.eval()
  b1_eo = b1_e.eval()
  w1_do = w1_d.eval()
  b1_do = b1_d.eval()

In [0]:
b1_do

In [0]:
plt.matshow(w1_eo.T);
plt.grid(False)
plt.colorbar()

In [0]:
plt.matshow(w1_do);
plt.grid(False)
plt.colorbar()

In [0]:
plt.matshow(np.matmul(w1_eo, w1_do));
plt.grid(False)
plt.colorbar()

In [0]:
plt.matshow(w1_eo.T-w1_do);
plt.grid(False)
plt.colorbar()

In [0]:
dec_mean_out = np.squeeze(np.array(dec_mean_out))
dec_mean_out_i1 = np.squeeze(np.array(dec_mean_out_i1))
enc_mean_out = np.squeeze(np.array(enc_mean_out))
dec_mean_out.shape

In [0]:
plt.scatter(enc_mean_out[:,0], enc_mean_out[:,1]);
plt.axis('equal');

In [0]:
plt.scatter(enc_mean_out[:,0], enc_mean_out[:,2])
plt.axis('equal');

In [0]:
enc_mean_out.shape

In [0]:
plt.matshow(np.matmul(enc_mean_out.T,enc_mean_out));
plt.grid(False)
plt.colorbar()

In [0]:
enc_shift = np.matmul(enc_mean_out.T,enc_mean_out)

In [0]:
from numpy import linalg as LA

In [0]:
w, v = LA.eigh(enc_shift)

In [0]:
w

In [0]:
v

In [0]:
np.max(enc_shift - np.einsum("ij, jk, km->im", v,np.diag(w), v.T))

In [0]:
enc_mean_out.shape

In [0]:
q.shape

In [0]:
r.shape

In [0]:
plt.matshow(np.matmul(q.T, q));
plt.grid(False)
plt.colorbar()

In [0]:
np.matmul(q.T, q)

In [0]:
r

In [0]:
np.linalg.inv(r)

In [0]:
plt.scatter(r[0,:], r[2,:]);
plt.axis('equal');

In [0]:
plt.scatter(r[0,:], r[1,:]);
plt.axis('equal');

In [0]:
q

In [0]:
q, r = np.linalg.qr(enc_mean_out.T)

In [0]:
rotated = np.einsum("ij, jk, km->im", v.T,enc_shift, v)

In [0]:
enc_mean_out_1 = np.matmul(enc_mean_out, v.T)

In [0]:
plt.scatter(enc_mean_out_1[:,0], enc_mean_out_1[:,2]);
plt.axis('equal');

In [0]:
import numpy as np
from scipy.linalg import qr

n = 3
H = np.random.randn(n, n)
Q, R = qr(H)

print (Q.dot(Q.T))

In [0]:
enc_mean_rd = np.matmul(enc_mean_out, Q)

In [0]:
plt.scatter(enc_mean_rd[:,0], enc_mean_rd[:,1]);
plt.axis('equal');

In [0]:
plt.scatter(enc_log_sd_out[:,0], enc_log_sd_out[:,1]);
plt.axis('equal');

In [0]:
plt.scatter(enc_log_sd_out[:,0], enc_log_sd_out[:,2]);
plt.axis('equal');

In [0]:
Z_plot = np.reshape(Z_test, [31, 31, 31, 3])
dc = np.reshape(dec_mean_out_i1, [31, 31, 31, 6])

In [0]:
plt.scatter(train[:,0], train[:,1], label='train_data');
plt.scatter(dec_mean_out[:,0], dec_mean_out[:,1], label='learnt model')
plt.scatter(dc[:,0,0,0], dc[:,0,0,1], label='changing z1');
plt.scatter(dc[0,:,0,0], dc[0,:,0,1], label='changing z2');
plt.scatter(dc[0,0,:,0], dc[0,0,:,1], label='changing z3');
plt.legend();

In [0]:
plt.scatter(train[:,2], train[:,3], label='train_data');
plt.scatter(dec_mean_out[:,2], dec_mean_out[:,3], label='learnt model')
plt.scatter(dc[:,0,0,2], dc[:,0,0,3], label='changing z1');
plt.scatter(dc[0,:,0,2], dc[0,:,0,3], label='changing z2');
plt.scatter(dc[0,0,:,2], dc[0,0,:,3], label='changing z3');
plt.legend();

In [0]:
plt.scatter(train[:,4], train[:,5], label='train_data');
plt.scatter(dec_mean_out[:,4], dec_mean_out[:,5], label='learnt model')
plt.scatter(dc[:,0,0,4], dc[:,0,0,5], label='changing z1');
plt.scatter(dc[0,:,0,4], dc[0,:,0,5], label='changing z2');
plt.scatter(dc[0,0,:,4], dc[0,0,:,5], label='changing z3');
plt.legend();

In [0]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

In [0]:
angle = np.pi/4
sin = np.sin(angle)
cos = np.cos(angle)
rot_z = np.array([[cos, -sin, 0],[sin, cos, 0],[0,0,1]])
rot_enc = np.matmul(enc_mean_out, rot_z)

In [0]:
#plot the training and test data 
import plotly.plotly as py
import numpy as np
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Contours, Histogram2dContour, Marker, Scatter

configure_plotly_browser_state()

init_notebook_mode(connected=False)



import plotly.graph_objs as go

# x, y, z = train[:,3], train[:,1], train[:,0]
# xt, yt, zt = train[:,4], train[:,2], train[:,5]

x, y, z = enc_mean_out[:,0], enc_mean_out[:,1], enc_mean_out[:,2]
x1, y1, z1 = rot_enc[:,0], rot_enc[:,1], rot_enc[:,2]

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    name = 'line',
    mode='markers',
    marker=dict(
        size=5,
        color=1,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)


trace2 = go.Scatter3d(
    x=x1,
    y=y1,
    z=z1,
    name = 'line',
    mode='markers',
    marker=dict(
        size=5,
        color=1,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)



fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='3d-scatter-colorscale')